In [ ]:
# fonctions disponibles : 
# spatial_interpolate, f , f_simple, f_ED, f_ED_invert, get_SReal, Inside, atPose, intersection, intersect, all_intersections

In [83]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import (StructField,StructType, BooleanType, DoubleType,LongType)
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, to_timestamp, unix_timestamp,udf
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType, FloatType, DataType, TimestampType, Row
import math
import inspect
from pyspark.sql.functions import lit

In [4]:
df_data = spark.read.option("header",True).option("inferSchema",True).csv("./GAST/VGP-week3-data.csv")

In [5]:
data_schema = df_data.schema
print(data_schema)

StructType(List(StructField(kit_id,IntegerType,true),StructField(participant_id,IntegerType,true),StructField(time,StringType,true),StructField(lat,DoubleType,true),StructField(lon,DoubleType,true),StructField(PM2.5,StringType,true),StructField(PM10,StringType,true),StructField(PM1.0,StringType,true),StructField(NO2,StringType,true),StructField(BC,StringType,true),StructField(activity,StringType,true),StructField(event,StringType,true)))


In [7]:
#Rename Colums 
df_data = df_data.withColumnRenamed('PM2.5','PM2_5')
df_data = df_data.withColumnRenamed('PM1.0','PM1_0')


#Change NULL string to null value 
df_data = df_data.withColumn(("BC"), when(col("BC") == "NULL", None).otherwise(col("BC")))
df_data = df_data.withColumn(("PM2_5"), when(col("PM2_5") == "NULL", None).otherwise(col("PM2_5")))
df_data = df_data.withColumn(("PM10"),  when(col("PM10") == "NULL",  None).otherwise(col("PM10")))
df_data = df_data.withColumn(("PM1_0"), when(col("PM1_0") == "NULL", None).otherwise(col("PM1_0")))
df_data = df_data.withColumn(("NO2"),   when(col("NO2") == "NULL",   None).otherwise(col("NO2")))
df_data = df_data.withColumn(("activity"), when(col("activity") == "NULL", None).otherwise(col("activity")))
df_data = df_data.withColumn(("event"), when(col("event") == "NULL", None).otherwise(col("event")))


In [8]:
df_data = df_data.withColumn('time', unix_timestamp('time','yyyy-MM-dd HH:mm:ss').alias('time'))

In [7]:
df_data.where(col('time').isNotNull()).show()
df_data.printSchema()

+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|      time|             lat|             lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|
+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|1573718400|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718410|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718420|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718430|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718440|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718450|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|

In [9]:
intime_schema = [StructField('val', FloatType(),True),
                 StructField('t1', IntegerType(),True)]
intime = StructType(fields=intime_schema)

Ureal_format = [StructField('a', IntegerType(),True),
              StructField('b', DoubleType(),True),
              StructField('c', DoubleType(),True),
              StructField('r', BooleanType(),True),
              StructField('t1', LongType(),True),
              StructField('t2', LongType(),True)]
UrealType = StructType(Ureal_format)

Position_format = [StructField('lat', DoubleType(),True),
              StructField('long', DoubleType(),True)]
PositionType = StructType(Position_format)

USreal_format = [StructField('a', IntegerType(),True),
              StructField('b', DoubleType(),True),
              StructField('c', DoubleType(),True),
              StructField('r', BooleanType(),True),
              StructField('lat1', DoubleType(),True),
              StructField('long1', DoubleType(),True),
              StructField('lat2', DoubleType(),True),
              StructField('long2', DoubleType(),True)]
USrealType = StructType(USreal_format)

Mreal_format = [StructField('time_units', ArrayType(UrealType),True)]
MrealType = StructType(Mreal_format)

MSreal_format = [StructField('space_units', ArrayType(USrealType),True)]
MSrealType = StructType(MSreal_format)

In [37]:
def spatial_interpolate(Key_serie_row, measures_row, longitude_row, latitude_row):
    usreal_list=[]
    for i in range(len(Key_serie_row)-2):
            a=0
            while (i<len(Key_serie_row)-2) and (Key_serie_row[i+1]-Key_serie_row[i]) == 0 :
                i=i+1
            
            b=(float(measures_row[i+1])-float(measures_row[i]))/(Key_serie_row[i+1]-Key_serie_row[i])
            c=((float(measures_row[i])*Key_serie_row[i+1])-(float(measures_row[i+1])*Key_serie_row[i]))/(Key_serie_row[i+1]-Key_serie_row[i])
            x1=[(latitude_row[i], longitude_row[i])]
            x2=[(latitude_row[i+1], longitude_row[i+1])]
            lat1 = latitude_row[i]
            long1 = longitude_row[i]
            lat2 = latitude_row[i+1]
            long2 = longitude_row[i+1]
            r=False
            new_usreal = [a,b,c,r,lat1,long1,lat2,long2]
            usreal_list.append(new_usreal)
    return Row(one_participant_profile = usreal_list)

In [154]:
from pyspark.sql.functions import sqrt

def f(longitude_row, latitude_row):
    key_list=[]
    for i in range(len(longitude_row)):
        key = math.sqrt((longitude_row[i]*longitude_row[i]) + (latitude_row[i]*latitude_row[i]))
        key_list.append(key)
    return key_list

def f_Simple(longitude_row, latitude_row):
    key = math.sqrt((longitude_row*longitude_row) + (latitude_row*latitude_row))
    return key

def f_ED(long, lat, precision_long):
    ent=int(long*pow(10,precision_long))*10
    if long < 0:
        ent+=1
    dec=lat/pow(10,3)
    if lat < 0:
        lat+=pow(10,-1)
    return ent+dec 
f_ED(2.1308233, 48.8014233,3)

21300.0488014233

In [157]:
def f_ED_invert(key,precision_long):
    ent = int(key)
    if ent % 10 == 1:
        ent = -ent 
    ent = int(ent/10)
    long = ent / pow(10,precision_long)
    dec = key - int(key) 
    if int(dec*10)==1:
        dec = -dec 
    dec = dec*10 - int(dec*10)
    lat = dec * pow(10,2)
    return long,lat 

f_ED_invert(f_ED(2.1308233, 48.8014233,3),3)
    
    

(2.13, 48.80142330148374)

In [13]:
test = df_data.filter(df_data["PM1_0"].isNotNull()).filter(df_data["lon"].isNotNull()).filter(df_data["lat"].isNotNull()).select("participant_id","lat","lon","PM1_0")

test = test.groupBy("participant_id").agg(F.collect_list(col('lat')).alias('lat'), F.collect_list(col('lon')).alias('lon'),F.collect_list(col("PM1_0")).alias("PM1_0"))

udfKey = udf(f, ArrayType(DoubleType()))
test_profiles = test.withColumn('keys', udfKey('lat','lon'))

In [14]:
test.show()

+--------------+--------------------+--------------------+--------------------+
|participant_id|                 lat|                 lon|               PM1_0|
+--------------+--------------------+--------------------+--------------------+
|       9999920|[48.8014233333333...|[2.13082333333333...|[19, 19, 16, 16, ...|
|       9999955|[48.8012116666667...|[2.13086166666667...|[3, 3, 3, 3, 3, 3...|
|       9999975|[48.8067434, 48.8...|[2.1306277, 2.132...|[25, 25, 21, 21, ...|
|       9999960|[48.8582383333333...|[2.34773166666667...|[10, 8, 8, 9, 9, ...|
|       9999964|[48.7717266666667...|[2.00586333333333...|[4, 4, 4, 4, 4, 4...|
|       9999962|[48.799435, 48.79...|[2.14138166666667...|[7, 7, 7, 7, 7, 7...|
+--------------+--------------------+--------------------+--------------------+



In [33]:
test_profiles.show()

+--------------+--------------------+--------------------+--------------------+--------------------+
|participant_id|                 lat|                 lon|               PM1_0|                keys|
+--------------+--------------------+--------------------+--------------------+--------------------+
|       9999920|[48.8014233333333...|[2.13082333333333...|[19, 19, 16, 16, ...|[48.8479204003311...|
|       9999955|[48.8012116666667...|[2.13086166666667...|[3, 3, 3, 3, 3, 3...|[48.8477106073281...|
|       9999975|[48.8067434, 48.8...|[2.1306277, 2.132...|[25, 25, 21, 21, ...|[48.8532268710005...|
|       9999960|[48.8582383333333...|[2.34773166666667...|[10, 8, 8, 9, 9, ...|[48.9146123056849...|
|       9999964|[48.7717266666667...|[2.00586333333333...|[4, 4, 4, 4, 4, 4...|[48.8129573961674...|
|       9999962|[48.799435, 48.79...|[2.14138166666667...|[7, 7, 7, 7, 7, 7...|[48.8463956885414...|
+--------------+--------------------+--------------------+--------------------+------------

In [34]:
list_keys = list(test_profiles.select('keys').head())[0]
list_mesure = list(test_profiles.select('PM1_0').head())[0]
list_lon = list(test_profiles.select('lon').head())[0]
list_lat = list(test_profiles.select('lat').head())[0]
#list_measure = list(PM100_test.select('PM1_0').head())[0]
result = spatial_interpolate (list_keys, list_mesure, list_lon, list_lat)


In [35]:
#print(result)
#result.collect()
toto = spark.createDataFrame(sc.parallelize([result]),MSrealType)
toto.show()

+--------------------+
|         space_units|
+--------------------+
|[[0, -0.0, 18.999...|
+--------------------+



In [38]:
udfProfilSpatial = udf(spatial_interpolate, MSrealType)
temp_spatial_profil = test_profiles.withColumn('profil_spatial', udfProfilSpatial("keys","PM1_0","lon","lat"))

In [39]:
temp_spatial_profil.show()

+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|participant_id|                 lat|                 lon|               PM1_0|                keys|      profil_spatial|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       9999920|[48.8014233333333...|[2.13082333333333...|[19, 19, 16, 16, ...|[48.8479204003311...|[[[0, -0.0, 18.99...|
|       9999955|[48.8012116666667...|[2.13086166666667...|[3, 3, 3, 3, 3, 3...|[48.8477106073281...|[[[0, -0.0, 3.0, ...|
|       9999975|[48.8067434, 48.8...|[2.1306277, 2.132...|[25, 25, 21, 21, ...|[48.8532268710005...|[[[0, 0.0, 24.999...|
|       9999960|[48.8582383333333...|[2.34773166666667...|[10, 8, 8, 9, 9, ...|[48.9146123056849...|[[[0, -15399.4219...|
|       9999964|[48.7717266666667...|[2.00586333333333...|[4, 4, 4, 4, 4, 4...|[48.8129573961674...|[[[0, -224826.516...|
|       9999962|[48.7994

In [41]:
temp_spatial_profil.printSchema()
def get_SReal(id_participant):
    toto = temp_spatial_profil.select('participant_id','profil_spatial')
    a = toto.filter(toto['participant_id'] == id_participant).select('profil_spatial')
    b = a.select(a['profil_spatial']['space_units'])
    c = b.collect()[0]
    maListe = c[0]
    tata = spark.createDataFrame(maListe,USrealType)
    return tata
get_SReal(9999920).show()

root
 |-- participant_id: integer (nullable = true)
 |-- lat: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lon: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- PM1_0: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keys: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- profil_spatial: struct (nullable = true)
 |    |-- space_units: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: integer (nullable = true)
 |    |    |    |-- b: double (nullable = true)
 |    |    |    |-- c: double (nullable = true)
 |    |    |    |-- r: boolean (nullable = true)
 |    |    |    |-- lat1: double (nullable = true)
 |    |    |    |-- long1: double (nullable = true)
 |    |    |    |-- lat2: double (nullable = true)
 |    |    |    |-- long2: double (nullable = true)

+---+------------------+-------------------+-----+----------------+---

In [112]:
def inside(profilSpatial, unPoint):
    profilSpatial = profilSpatial.withColumn(("maxLat"), when(col("lat1") <= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("maxLong"), when(col("long1") <= col('long2'), col('long2')).otherwise(col("long1")))
    profilSpatial = profilSpatial.withColumn(("minLat"), when(col("lat1") >= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("minLong"), when(col("long1") >= col('long2'), col('long2')).otherwise(col("long1")))
    result = profilSpatial.filter(((profilSpatial['lat2'] - profilSpatial['lat1']) / (profilSpatial['long2'] - profilSpatial['long1'])) == ((unPoint.head()['lat']-profilSpatial['lat1']) / (unPoint.head()['long']-profilSpatial['long1']))).filter(
    unPoint.head()['lat'] <= profilSpatial['maxLat']).filter(unPoint.head()['lat'] >= profilSpatial['minLat']).filter(
    unPoint.head()['long'] <= profilSpatial['maxLong']).filter(unPoint.head()['long'] >= profilSpatial['minLong'])
    if(result.rdd.isEmpty()):
        return 0
    else: 
        return 1


1

In [113]:
# Test inside()
pointTest = [48.801415, 2.130805]
sPOINT = spark.createDataFrame(sc.parallelize([pointTest]),PositionType)
inside(get_SReal(9999920), sPOINT)

1

In [138]:
intime_schema = [StructField('val', DoubleType(),True),
                 StructField('Lat', DoubleType(),True),
                 StructField('Long', DoubleType(),True)]
intimeSpatial = StructType(fields=intime_schema)

In [144]:
def atPos(profilSpatial, unPoint):
    
    profilSpatial = profilSpatial.withColumn(("maxLat"), when(col("lat1") <= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("maxLong"), when(col("long1") <= col('long2'), col('long2')).otherwise(col("long1")))
    profilSpatial = profilSpatial.withColumn(("minLat"), when(col("lat1") >= col('lat2'), col('lat2')).otherwise(col("lat1")))
    profilSpatial = profilSpatial.withColumn(("minLong"), when(col("long1") >= col('long2'), col('long2')).otherwise(col("long1")))
    result = profilSpatial.filter(((profilSpatial['lat2'] - profilSpatial['lat1']) / (profilSpatial['long2'] - profilSpatial['long1'])) == ((unPoint.head()['lat']-profilSpatial['lat1']) / (unPoint.head()['long']-profilSpatial['long1']))).filter(
    unPoint.head()['lat'] <= profilSpatial['maxLat']).filter(unPoint.head()['lat'] >= profilSpatial['minLat']).filter(
    unPoint.head()['long'] <= profilSpatial['maxLong']).filter(unPoint.head()['long'] >= profilSpatial['minLong'])
    if(result.rdd.isEmpty()):
        return "Le point n'est pas sur notre trajectoire"
    else: 
        #lattList_NotrePoint = [unPoint[0]]
        #longList_NotrePoint = [unPoint[1]]
        val = (result.head()['a'] * f_Simple(unPoint.head()['long'],unPoint.head()['lat'])* f_Simple(unPoint.head()['long'],unPoint.head()['lat'])) + (result.head()['b'] * f_Simple(unPoint.head()['long'],unPoint.head()['lat'])) + result.head()['c'] 
        r = [(val, unPoint.head()['lat'], unPoint.head()['long'])]
        resultFinal = spark.createDataFrame(r, intimeSpatial)
        return r


In [145]:
# Test atPos()
pointTest = [48.801415, 2.130805]
sPOINT = spark.createDataFrame(sc.parallelize([pointTest]),PositionType)
atPos(get_SReal(9999920), sPOINT)

[(18.999999999221334, 48.801415, 2.130805)]

In [140]:
def intersection(USreal1,USreal2):
    segment1= [USreal1.__getattr__('lat1'),USreal1.__getattr__('long1'),USreal1.__getattr__('lat2'),USreal1.__getattr__('long2')]
    segment2= [USreal2.__getattr__('lat1'),USreal2.__getattr__('long1'),USreal2.__getattr__('lat2'),USreal2.__getattr__('long2')]
    if (segment1[0]!=segment1[2]):
        a1,b1=(segment1[1]-segment1[3])/(segment1[0]-segment1[2]),segment1[1]-((segment1[1]-segment1[3])/(segment1[0]-segment1[2]))*segment1[0]
    else : 
        a1,b1 = 999999999999,segment2[1]-999999999999*segment2[0]    
    if (segment2[0]!=segment2[2]):
        a2,b2=(segment2[1]-segment2[3])/(segment2[0]-segment2[2]),segment2[1]-((segment2[1]-segment2[3])/(segment2[0]-segment2[2]))*segment2[0]
    else :
        a2,b2 = 999999999999,segment2[1]-999999999999*segment2[0]   
    if (a1 != a2):
        x0 = (b2-b1)/(a1-a2)
        y0 = (a1*b2-a2*b1)/(a1-a2)
        if (x0>=max(min(segment1[0],segment1[2]),min(segment2[0],segment2[2]))) and (x0<=min(max(segment1[0],segment1[2]),max(segment1[0],segment1[2]))):
            return (x0,y0)
        else:
            return "intersection_hors_segment"
    elif b1==b2:
        return "segment_identique", [max(min(segment1[0],segment1[2]),min(segment2[0],segment2[2])),min(max(segment1[0],segment1[2]),max(segment1[0],segment1[2]))]
    else:
        return "intersection_impossible" 
            
def intersect(MSreal_row1, MSreal_row2):
    list_MSreal1=MSreal_row1.__getattr__('profil_spatial').__getattr__("space_units")
    list_MSreal2=MSreal_row2.__getattr__('profil_spatial').__getattr__("space_units")
    for seg1 in list_MSreal1:
        for seg2 in list_MSreal2:
            if intersection(seg1,seg2) != "intersection_hors_segment" and  intersection(seg1,seg2) != "intersection_impossible" :
                return 1
def all_intersections(MSreal_row1, MSreal_row2):
    list_MSreal1=MSreal_row1.__getattr__('profil_spatial').__getattr__("space_units")
    list_MSreal2=MSreal_row2.__getattr__('profil_spatial').__getattr__("space_units")
    list_intersect=[]
    for seg1 in list_MSreal1:
        for seg2 in list_MSreal2:
            if intersection(seg1,seg2) != "intersection_hors_segment" and  intersection(seg1,seg2) != "intersection_impossible" :
                list_intersect.append(intersection(seg1,seg2))
    return list_intersect

In [61]:
temp_spatial_profil.printSchema()

root
 |-- participant_id: integer (nullable = true)
 |-- lat: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lon: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- PM1_0: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keys: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- profil_spatial: struct (nullable = true)
 |    |-- space_units: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- a: integer (nullable = true)
 |    |    |    |-- b: double (nullable = true)
 |    |    |    |-- c: double (nullable = true)
 |    |    |    |-- r: boolean (nullable = true)
 |    |    |    |-- lat1: double (nullable = true)
 |    |    |    |-- long1: double (nullable = true)
 |    |    |    |-- lat2: double (nullable = true)
 |    |    |    |-- long2: double (nullable = true)



In [124]:
MSR1= temp_spatial_profil.filter(temp_spatial_profil['participant_id']== 9999920).select('profil_spatial').rdd.collect()[0]
MSR2= temp_spatial_profil.filter(temp_spatial_profil['participant_id']== 9999955).select('profil_spatial').rdd.collect()[0]
MSR1

Row(profil_spatial=Row(space_units=[Row(a=0, b=-0.0, c=18.999999999221334, r=False, lat1=48.8014233333333, long1=2.13082333333333, lat2=48.801415, long2=2.130805), Row(a=0, b=141443.85083696034, c=-6909217.676107097, r=False, lat1=48.801415, long1=2.130805, lat2=48.8013933333333, long2=2.130815), Row(a=0, b=0.0, c=16.0, r=False, lat1=48.8013933333333, long1=2.130815, lat2=48.8014166666667, long2=2.13086833333333), Row(a=0, b=0.0, c=16.0, r=False, lat1=48.8014166666667, long1=2.13086833333333, lat2=48.80145, long2=2.13094166666667), Row(a=0, b=-0.0, c=16.0, r=False, lat1=48.80145, long1=2.13094166666667, lat2=48.801365, long2=2.13087333333333), Row(a=0, b=-34459.94141639987, c=1683310.5422193715, r=False, lat1=48.801365, long1=2.13087333333333, lat2=48.8013966666667, long2=2.13081333333333), Row(a=0, b=-34459.94141639987, c=1683310.5422193715, r=False, lat1=48.801365, long1=2.13087333333333, lat2=48.8013966666667, long2=2.13081333333333), Row(a=0, b=-34459.94141639987, c=1683310.5422193

In [146]:
#tester la fonction intersection pour deux segment USReal1 et USReal2 d'un seule MSReal1
USR1=MSR1.__getattr__('profil_spatial').__getattr__("space_units")[0]
USR2=MSR1.__getattr__('profil_spatial').__getattr__("space_units")[1]
#tester sur deux segments dont l'intersection est (1.5,1.5) (facile à manipuler pour les tests)
x1 = Row(lat1=1,long1=1,lat2=2,long2=2)
x2 = Row(lat1=1,long1=2,lat2=2,long2=1)
intersection(USR1,USR2),intersection(x1,x2)

((48.801415, 2.130804999999998), (1.5, 1.5))

In [147]:
#segment identique 
intersection(USR1,USR1)

('segment_identique', [48.801415, 48.8014233333333])

In [148]:
#tester le cas d'une intersection hors segment
x1 = Row(lat1=1,long1=1,lat2=2,long2=2)
x2 = Row(lat1=1,long1=3,lat2=2,long2=3)
intersection(x1,x2)

'intersection_hors_segment'

In [134]:
#tester si MSreal1 et MSreal2 ont des trajectoires qui s'intersectent ou pas 
intersect(MSR1,MSR2)

1

In [141]:
# trouver toutes les intersection entre MSreal1 et MSreal2 
# ! à ne pas réexecuter : sa prends énormément de temps à cause de la taille des données et l'absence d'un index
all_intersections(MSR1,MSR2)

[(48.801415000000134, 2.1308050000002914),
 (48.80142164185834, 2.1308196120884038),
 (48.80142079710127, 2.1308177536228667),
 (48.801417380952664, 2.130810238095872),
 (48.801415677506746, 2.1308064905148343),
 (48.80142146067411, 2.130819213483111),
 (48.80142146067411, 2.130819213483111),
 (48.801419817275736, 2.130815598006652),
 (48.801419817275736, 2.130815598006652),
 (48.801418990610316, 2.1308137793427315),
 (48.80141927295919, 2.130814400510249),
 (48.801420313351485, 2.1308166893733254),
 (48.801416436781615, 2.130808160919548),
 (48.801421202588116, 2.1308186456939073),
 (48.801421202588116, 2.1308186456939073),
 (48.801421202588116, 2.1308186456939073),
 (48.80142183760699, 2.130820042735433),
 (48.8014185367545, 2.130812780859922),
 (48.8014185367545, 2.130812780859922),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (48.801415185402, 2.1308054078843837),
 (4

In [153]:
pow(10,-1)

0.1